## A Portal to AI - Human Role Playing Experiences
###    Creating a Mission-Driven NPC Chatbot for RPGs

In [ ]:
# Install required packages.
!pip install flask pyngrok google-generativeai nltk

# Import libraries.
from flask import Flask, request, jsonify
import google.generativeai as genai
from pyngrok import ngrok
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import threading
import json
import sqlite3
import datetime
import os
import time
import requests

# Initialize Flask app.
app = Flask(__name__)

# Database setup.
DB_PATH = 'bloodhound_mission.db'

def init_database():
    """Initialize the SQLite database with required tables"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Users table.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT UNIQUE NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            last_active TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    # Mission progress table.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mission_progress (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            current_objective INTEGER DEFAULT 0,
            completed_objectives TEXT DEFAULT '[]',
            items TEXT DEFAULT '[]',
            mission_data TEXT DEFAULT '{}',
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users (id)
        )
    ''')

    # Chat history table.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            message TEXT NOT NULL,
            response TEXT NOT NULL,
            sentiment_score REAL,
            objective_at_time INTEGER,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users (id)
        )
    ''')

    # Mission statistics table.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mission_stats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            total_messages INTEGER DEFAULT 0,
            objectives_completed INTEGER DEFAULT 0,
            missions_started INTEGER DEFAULT 1,
            missions_completed INTEGER DEFAULT 0,
            total_playtime_minutes INTEGER DEFAULT 0,
            FOREIGN KEY (user_id) REFERENCES users (id)
        )
    ''')

    conn.commit()
    conn.close()
    print("✓ Database initialized successfully")

def get_or_create_user(name):
    """Get existing user or create new one"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Try to get existing user.
    cursor.execute('SELECT id FROM users WHERE name = ?', (name,))
    user = cursor.fetchone()

    if user:
        user_id = user[0]
        # Update last active.
        cursor.execute('UPDATE users SET last_active = CURRENT_TIMESTAMP WHERE id = ?', (user_id,))
    else:
        # Create new user.
        cursor.execute('INSERT INTO users (name) VALUES (?)', (name,))
        user_id = cursor.lastrowid

        # Initialize mission progress.
        cursor.execute('''
            INSERT INTO mission_progress (user_id, current_objective, completed_objectives, items, mission_data)
            VALUES (?, 0, '[]', '[]', '{}')
        ''', (user_id,))

        # Initialize stats.
        cursor.execute('INSERT INTO mission_stats (user_id) VALUES (?)', (user_id,))

    conn.commit()
    conn.close()
    return user_id

def load_user_data(user_id):
    """Load user's mission progress and data"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT current_objective, completed_objectives, items, mission_data
        FROM mission_progress WHERE user_id = ?
    ''', (user_id,))

    result = cursor.fetchone()
    conn.close()

    if result:
        return {
            'currentObjective': result[0],
            'completedObjectives': json.loads(result[1]),
            'items': json.loads(result[2]),
            'missionData': json.loads(result[3])
        }
    else:
        return {
            'currentObjective': 0,
            'completedObjectives': [],
            'items': [],
            'missionData': {}
        }

def save_user_data(user_id, data):
    """Save user's mission progress"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        UPDATE mission_progress
        SET current_objective = ?, completed_objectives = ?, items = ?, mission_data = ?, updated_at = CURRENT_TIMESTAMP
        WHERE user_id = ?
    ''', (
        data['currentObjective'],
        json.dumps(data['completedObjectives']),
        json.dumps(data['items']),
        json.dumps(data.get('missionData', {})),
        user_id
    ))

    conn.commit()
    conn.close()

def save_chat_interaction(user_id, message, response, sentiment_score, current_objective):
    """Save chat interaction to history"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO chat_history (user_id, message, response, sentiment_score, objective_at_time)
        VALUES (?, ?, ?, ?, ?)
    ''', (user_id, message, response, sentiment_score, current_objective))

    # Update stats.
    cursor.execute('''
        UPDATE mission_stats
        SET total_messages = total_messages + 1
        WHERE user_id = ?
    ''', (user_id,))

    conn.commit()
    conn.close()

def get_user_stats(user_id):
    """Get user statistics"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT total_messages, objectives_completed, missions_started, missions_completed
        FROM mission_stats WHERE user_id = ?
    ''', (user_id,))

    result = cursor.fetchone()
    conn.close()

    if result:
        return {
            'totalMessages': result[0],
            'objectivesCompleted': result[1],
            'missionsStarted': result[2],
            'missionsCompleted': result[3]
        }
    else:
        return {'totalMessages': 0, 'objectivesCompleted': 0, 'missionsStarted': 0, 'missionsCompleted': 0}

# Initialize database.
init_database()

# Download VADER lexicon for sentiment analysis.
try:
    nltk.download('vader_lexicon')
    sid = SentimentIntensityAnalyzer()
except:
    print("Warning: Could not download VADER lexicon")
    sid = None

# Set up Google Gemini (replace with your API key).
API_KEY = "YOUR_KEY_HERE"  # Replace with your actual API key.
genai.configure(api_key=API_KEY)

# Try different model names if this doesn't work.
try:
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("✓ Model initialized successfully")
except Exception as e:
    print(f"✗ Error initializing model: {e}")
    # Try alternative model names.
    try:
        model = genai.GenerativeModel('gemini-pro')
        print("✓ Model initialized with gemini-pro")
    except Exception as e2:
        print(f"✗ Error with gemini-pro: {e2}")
        model = None

# Test the API connection.
def test_api():
    if model:
        try:
            test_response = model.generate_content("Say 'API working'")
            print(f"✓ API Test Response: {test_response.text}")
            return True
        except Exception as e:
            print(f"✗ API Test Failed: {e}")
            return False
    return False

# API endpoint to get user data.
@app.route('/api/user/<username>')
def get_user_data(username):
    try:
        user_id = get_or_create_user(username)
        mission_progress = load_user_data(user_id)
        stats = get_user_stats(user_id)

        return jsonify({
            'missionProgress': mission_progress,
            'stats': stats
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Chat endpoint.
@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        if not data:
            return jsonify({'error': 'No JSON data received'}), 400

        message = data.get('message', '').strip()
        memory = data.get('memory', {})

        if not message:
            return jsonify({'error': 'No message provided'}), 400

        print(f"Received message: '{message}'")
        print(f"Current memory: {memory}")

        # Get or create user.
        user_name = memory.get('name', 'Unknown')
        user_id = get_or_create_user(user_name)

        # Load fresh data from database.
        user_data = load_user_data(user_id)
        memory.update(user_data)  # Update with latest from DB.

        # Analyze sentiment.
        sentiment_score = 0
        if sid:
            try:
                sentiment_score = sid.polarity_scores(message)['compound']
            except:
                sentiment_score = 0

        # Build conversation context.
        objectives = [
            "Break into Titanium Towers via West Tower basement subway entrance",
            "Get past basement guards and access elevator shaft",
            "Climb interior ladder to 110th floor and exit shaft",
            "Cross bridge from West Tower to East Tower",
            "Stealthily reach 121st floor penthouse in East Tower",
            "Locate and capture Dr. Eva from celebration party",
            "Return to West Tower with Dr. Eva",
            "Get Dr. Eva to basement and onto pickup rail car",
            "Stay on rail car until rendezvous point",
            "Meet pickup team at rendezvous for extraction",
            "Lead pursuit out of the city",
            "Shake all pursuit and head to headquarters"
        ]

        current_obj = memory.get('currentObjective', 0)
        completed = memory.get('completedObjectives', [])

        # Build prompt.
        prompt = f"""You are Lieutenant, a gruff cyberpunk military commander from the year 2087. You're briefing {user_name} on a stealth mission to capture Dr. Eva from Titanium Towers.

Current Mission Status:
- Current Objective: {current_obj + 1}/12 - {objectives[current_obj] if current_obj < len(objectives) else 'Mission Complete'}
- Completed Objectives: {len(completed)}/12
- Player Message: "{message}"
- Player Sentiment: {sentiment_score:.2f}

Mission Objectives:
{chr(10).join([f"{i+1}. {'✓' if i in completed else '►' if i == current_obj else '○'} {obj}" for i, obj in enumerate(objectives)])}

Respond as Lieutenant - be gruff but helpful. Use cyberpunk slang. If the player seems to be progressing or asking about the next step, you can advance their objective. Keep responses under 100 words."""

        print(f"Prompt: {prompt}")

        # Generate response with Google Gemini.
        try:
            print("Calling model.generate_content...")
            response = model.generate_content(prompt)
            print(f"Raw model response: {response}")

            if hasattr(response, 'text') and response.text:
                response_text = response.text.strip()
                print(f"Response text: '{response_text}'")

                # Simple objective progression logic.
                progress_keywords = ['done', 'completed', 'finished', 'next', 'ready', 'got it', 'success']
                if any(keyword in message.lower() for keyword in progress_keywords) and current_obj < len(objectives):
                    if current_obj not in completed:
                        completed.append(current_obj)
                        memory['completedObjectives'] = completed
                        memory['currentObjective'] = min(current_obj + 1, len(objectives) - 1)

                        # Update database.
                        save_user_data(user_id, memory)

                        # Update stats.
                        conn = sqlite3.connect(DB_PATH)
                        cursor = conn.cursor()
                        cursor.execute('UPDATE mission_stats SET objectives_completed = ? WHERE user_id = ?',
                                     (len(completed), user_id))
                        conn.commit()
                        conn.close()

                # Save chat interaction.
                save_chat_interaction(user_id, message, response_text, sentiment_score, current_obj)

                # Get updated stats.
                stats = get_user_stats(user_id)

                return jsonify({
                    'response': response_text,
                    'memory': memory,
                    'stats': stats
                })
            else:
                print("ERROR: No text in model response")
                return jsonify({'error': 'No response text generated'}), 500

        except Exception as model_error:
            print(f"ERROR in model generation: {model_error}")
            return jsonify({'error': f'Model error: {str(model_error)}'}), 500

    except Exception as e:
        print(f"ERROR in chat endpoint: {e}")
        return jsonify({'error': f'Server error: {str(e)}'}), 500

# Serve the frontend.
@app.route('/')
def index():
    return '''
<html>
<head>
    <title>Bloodhound Mission - Chatbot with 3D Avatar</title>
    <script src="https://cdn.jsdelivr.net/npm/three@0.132.2/build/three.min.js"></script>
    <style>
        body {
            background-color: #1a1a1a;
            font-family: 'Courier New', monospace;
            color: #00ffcc;
            margin: 20px;
        }
        #chat {
            border: 1px solid #00ffcc;
            padding: 10px;
            height: 300px;
            overflow-y: scroll;
            background-color: #333;
            flex-grow: 1;
        }
        #avatar {
            width: 200px;
            height: 200px;
            margin-right: 20px;
            border: 2px solid #00ffcc;
            border-radius: 10px;
            background: linear-gradient(135deg, #001122, #002244);
            box-shadow: 0 0 20px rgba(0, 255, 204, 0.3);
        }
        #mission-status {
            width: 300px;
            border: 1px solid #ff6600;
            background-color: #2a1a0a;
            padding: 10px;
            margin-left: 20px;
            height: 300px;
            overflow-y: auto;
        }
        #mission-status h3 {
            color: #ff6600;
            margin-top: 0;
            border-bottom: 1px solid #ff6600;
            padding-bottom: 5px;
        }
        .objective {
            margin: 8px 0;
            padding: 5px;
            border-left: 3px solid #666;
            font-size: 12px;
        }
        .objective.current {
            border-left-color: #ffff00;
            background-color: #333300;
            color: #ffff00;
        }
        .objective.completed {
            border-left-color: #00ff00;
            background-color: #003300;
            color: #00ff88;
            text-decoration: line-through;
        }
        .objective.pending {
            border-left-color: #666;
            background-color: #222;
            color: #888;
        }
        #user-stats {
            margin-top: 10px;
            padding: 8px;
            background-color: #1a1a2e;
            border: 1px solid #0066ff;
            font-size: 11px;
        }
        #user-stats h4 {
            color: #0066ff;
            margin: 0 0 5px 0;
        }
        .stat-item {
            color: #88ccff;
            margin: 2px 0;
        }
        input {
            width: 70%;
            padding: 10px;
            background-color: #444;
            color: #00ffcc;
            border: 1px solid #00ffcc;
        }
        button {
            padding: 10px;
            background-color: #00ffcc;
            color: #000;
            border: none;
            cursor: pointer;
        }
        #debug {
            background-color: #222;
            color: #ff6666;
            padding: 10px;
            margin-top: 10px;
            font-size: 12px;
            max-height: 100px;
            overflow-y: auto;
        }
    </style>
</head>
<body>
    <h1>🎯 BLOODHOUND MISSION - Chat with Lieutenant</h1>
    <div style="display: flex;">
        <div id="avatar"></div>
        <div id="chat"></div>
        <div id="mission-status">
            <h3>🎯 MISSION STATUS</h3>
            <div><strong>OBJECTIVE:</strong> Capture Dr. Eva alive</div>
            <div id="objectives-list"></div>
            <div id="user-stats">
                <h4>📊 STATS</h4>
                <div id="stats-content">Loading...</div>
            </div>
        </div>
    </div>
    <input type="text" id="message" placeholder="Type your message" onkeypress="if(event.key==='Enter') sendMessage()">
    <button onclick="sendMessage()">Send</button>
    <div id="debug">Debug info will appear here...</div>

    <script>
        // Debug function
        function addDebug(message) {
            const debugDiv = document.getElementById('debug');
            debugDiv.innerHTML += '<div>' + new Date().toLocaleTimeString() + ': ' + message + '</div>';
            debugDiv.scrollTop = debugDiv.scrollHeight;
        }

        // 3D Cyberpunk Avatar Setup
        const scene = new THREE.Scene();
        scene.background = new THREE.Color(0x0a0a0a);

        const camera = new THREE.PerspectiveCamera(75, 1, 0.1, 1000);
        const renderer = new THREE.WebGLRenderer({ antialias: true, alpha: true });
        renderer.setSize(200, 200);
        renderer.shadowMap.enabled = true;
        renderer.shadowMap.type = THREE.PCFSoftShadowMap;
        document.getElementById('avatar').appendChild(renderer.domElement);

        // Lighting setup
        const ambientLight = new THREE.AmbientLight(0x404040, 0.4);
        scene.add(ambientLight);

        const directionalLight = new THREE.DirectionalLight(0x00ffcc, 0.8);
        directionalLight.position.set(2, 2, 5);
        directionalLight.castShadow = true;
        scene.add(directionalLight);

        const pointLight = new THREE.PointLight(0xff3300, 0.6, 10);
        pointLight.position.set(-2, 1, 2);
        scene.add(pointLight);

        // Create cyberpunk humanoid
        const avatar = new THREE.Group();

        // Materials
        const bodyMaterial = new THREE.MeshPhongMaterial({
            color: 0x2a2a2a,
            shininess: 100,
            transparent: true,
            opacity: 0.9
        });

        const glowMaterial = new THREE.MeshPhongMaterial({
            color: 0x00ffcc,
            emissive: 0x003333,
            shininess: 100,
            transparent: true,
            opacity: 0.8
        });

        const eyeMaterial = new THREE.MeshPhongMaterial({
            color: 0xff0033,
            emissive: 0x660011,
            shininess: 200
        });

        // Head
        const headGeometry = new THREE.SphereGeometry(0.4, 16, 16);
        const head = new THREE.Mesh(headGeometry, bodyMaterial);
        head.position.y = 1.2;
        head.castShadow = true;
        avatar.add(head);

        // Eyes (glowing)
        const eyeGeometry = new THREE.SphereGeometry(0.08, 8, 8);
        const leftEye = new THREE.Mesh(eyeGeometry, eyeMaterial);
        leftEye.position.set(-0.15, 1.25, 0.3);
        const rightEye = new THREE.Mesh(eyeGeometry, eyeMaterial);
        rightEye.position.set(0.15, 1.25, 0.3);
        avatar.add(leftEye);
        avatar.add(rightEye);

        // Helmet/visor effect
        const visorGeometry = new THREE.SphereGeometry(0.42, 16, 8, 0, Math.PI * 2, 0, Math.PI * 0.6);
        const visorMaterial = new THREE.MeshPhongMaterial({
            color: 0x001122,
            transparent: true,
            opacity: 0.3,
            side: THREE.DoubleSide
        });
        const visor = new THREE.Mesh(visorGeometry, visorMaterial);
        visor.position.y = 1.3;
        avatar.add(visor);

        // Body (torso)
        const torsoGeometry = new THREE.CylinderGeometry(0.3, 0.35, 0.8, 8);
        const torso = new THREE.Mesh(torsoGeometry, bodyMaterial);
        torso.position.y = 0.4;
        torso.castShadow = true;
        avatar.add(torso);

        // Chest panel (glowing)
        const chestGeometry = new THREE.BoxGeometry(0.4, 0.3, 0.05);
        const chestPanel = new THREE.Mesh(chestGeometry, glowMaterial);
        chestPanel.position.set(0, 0.5, 0.32);
        avatar.add(chestPanel);

        // Arms
        const armGeometry = new THREE.CylinderGeometry(0.08, 0.1, 0.6, 8);
        const leftArm = new THREE.Mesh(armGeometry, bodyMaterial);
        leftArm.position.set(-0.45, 0.3, 0);
        leftArm.rotation.z = 0.3;
        leftArm.castShadow = true;
        avatar.add(leftArm);

        const rightArm = new THREE.Mesh(armGeometry, bodyMaterial);
        rightArm.position.set(0.45, 0.3, 0);
        rightArm.rotation.z = -0.3;
        rightArm.castShadow = true;
        avatar.add(rightArm);

        // Shoulder pads (cyberpunk style)
        const shoulderGeometry = new THREE.BoxGeometry(0.2, 0.15, 0.25);
        const leftShoulder = new THREE.Mesh(shoulderGeometry, glowMaterial);
        leftShoulder.position.set(-0.35, 0.65, 0);
        const rightShoulder = new THREE.Mesh(shoulderGeometry, glowMaterial);
        rightShoulder.position.set(0.35, 0.65, 0);
        avatar.add(leftShoulder);
        avatar.add(rightShoulder);

        // Legs
        const legGeometry = new THREE.CylinderGeometry(0.1, 0.12, 0.8, 8);
        const leftLeg = new THREE.Mesh(legGeometry, bodyMaterial);
        leftLeg.position.set(-0.15, -0.4, 0);
        leftLeg.castShadow = true;
        avatar.add(leftLeg);

        const rightLeg = new THREE.Mesh(legGeometry, bodyMaterial);
        rightLeg.position.set(0.15, -0.4, 0);
        rightLeg.castShadow = true;
        avatar.add(rightLeg);

        // Feet
        const footGeometry = new THREE.BoxGeometry(0.2, 0.1, 0.3);
        const leftFoot = new THREE.Mesh(footGeometry, bodyMaterial);
        leftFoot.position.set(-0.15, -0.85, 0.05);
        const rightFoot = new THREE.Mesh(footGeometry, bodyMaterial);
        rightFoot.position.set(0.15, -0.85, 0.05);
        avatar.add(leftFoot);
        avatar.add(rightFoot);

        // Cybernetic implants/details
        const implantGeometry = new THREE.SphereGeometry(0.05, 6, 6);
        const implantMaterial = new THREE.MeshPhongMaterial({
            color: 0xff6600,
            emissive: 0x331100,
            shininess: 200
        });

        const leftImplant = new THREE.Mesh(implantGeometry, implantMaterial);
        leftImplant.position.set(-0.3, 1.1, 0.25);
        const rightImplant = new THREE.Mesh(implantGeometry, implantMaterial);
        rightImplant.position.set(0.3, 1.1, 0.25);
        avatar.add(leftImplant);
        avatar.add(rightImplant);

        // Add wireframe overlay for extra cyberpunk effect
        const wireframeGeometry = new THREE.SphereGeometry(0.41, 8, 8);
        const wireframeMaterial = new THREE.MeshBasicMaterial({
            color: 0x00ffcc,
            wireframe: true,
            transparent: true,
            opacity: 0.2
        });
        const wireframeHead = new THREE.Mesh(wireframeGeometry, wireframeMaterial);
        wireframeHead.position.y = 1.2;
        avatar.add(wireframeHead);

        scene.add(avatar);
        camera.position.set(0, 0.5, 3);
        camera.lookAt(0, 0.5, 0);

        // Animation variables
        let time = 0;
        let talking = false;
        let pulseIntensity = 0;

        // Enhanced animation function
        function animate() {
            requestAnimationFrame(animate);
            time += 0.02;

            // Idle breathing animation
            avatar.scale.y = 1 + Math.sin(time * 1.5) * 0.02;

            // Head movement
            head.rotation.y = Math.sin(time * 0.8) * 0.1;
            head.rotation.x = Math.sin(time * 0.6) * 0.05;

            // Glowing effects
            pulseIntensity = (Math.sin(time * 3) + 1) * 0.5;
            chestPanel.material.emissive.setHSL(0.5, 1, pulseIntensity * 0.2);
            leftShoulder.material.emissive.setHSL(0.5, 1, pulseIntensity * 0.15);
            rightShoulder.material.emissive.setHSL(0.5, 1, pulseIntensity * 0.15);

            // Eye glow
            leftEye.material.emissive.setHSL(0.95, 1, pulseIntensity * 0.3);
            rightEye.material.emissive.setHSL(0.95, 1, pulseIntensity * 0.3);

            // Implant glow
            leftImplant.material.emissive.setHSL(0.08, 1, pulseIntensity * 0.2);
            rightImplant.material.emissive.setHSL(0.08, 1, pulseIntensity * 0.2);

            // Talking animation
            if (talking) {
                head.scale.y = 1 + Math.sin(time * 10) * 0.05;
                chestPanel.material.emissive.multiplyScalar(1.5);
            } else {
                head.scale.y = 1;
            }

            // Slow rotation for dramatic effect
            avatar.rotation.y += 0.005;

            renderer.render(scene, camera);
        }
        animate();

        // Function to trigger talking animation
        window.setAvatarTalking = function(isTalking) {
            talking = isTalking;
        };

        // Initialize memory
        let memory;
        let userName = '';
        let userStats = {};

        // Mission objectives
        const objectives = [
            "Break into Titanium Towers via West Tower basement subway entrance",
            "Get past basement guards and access elevator shaft",
            "Climb interior ladder to 110th floor and exit shaft",
            "Cross bridge from West Tower to East Tower",
            "Stealthily reach 121st floor penthouse in East Tower",
            "Locate and capture Dr. Eva from celebration party",
            "Return to West Tower with Dr. Eva",
            "Get Dr. Eva to basement and onto pickup rail car",
            "Stay on rail car until rendezvous point",
            "Meet pickup team at rendezvous for extraction",
            "Lead pursuit out of the city",
            "Shake all pursuit and head to headquarters"
        ];

        // Get username first
        userName = prompt("What's your name, chummer?") || "Unknown";

        async function loadUserData() {
            try {
                const response = await fetch(`/api/user/${userName}`);
                const userData = await response.json();

                if (userData.missionProgress) {
                    memory = {
                        name: userName,
                        currentObjective: userData.missionProgress.currentObjective,
                        completedObjectives: userData.missionProgress.completedObjectives,
                        items: userData.missionProgress.items,
                        missionData: userData.missionProgress.missionData
                    };
                    userStats = userData.stats;
                    addDebug('Loaded user data from database');
                } else {
                    memory = {
                        name: userName,
                        currentObjective: 0,
                        completedObjectives: [],
                        items: [],
                        missionData: {}
                    };
                    userStats = {totalMessages: 0, objectivesCompleted: 0, missionsStarted: 1, missionsCompleted: 0};
                }

                updateMissionStatus();
                updateUserStats();

            } catch (e) {
                addDebug('Error loading from database: ' + e.message);
                memory = {
                    name: userName,
                    currentObjective: 0,
                    completedObjectives: [],
                    items: [],
                    missionData: {}
                };
                updateMissionStatus();
            }
        }

        function updateUserStats() {
            const statsContent = document.getElementById('stats-content');
            statsContent.innerHTML = `
                <div class="stat-item">Messages: ${userStats.totalMessages || 0}</div>
                <div class="stat-item">Objectives: ${userStats.objectivesCompleted || 0}/12</div>
                <div class="stat-item">Missions: ${userStats.missionsCompleted || 0}</div>
            `;
        }

        function updateMissionStatus() {
            const objectivesList = document.getElementById('objectives-list');
            objectivesList.innerHTML = '';

            objectives.forEach((objective, index) => {
                const div = document.createElement('div');
                div.className = 'objective';

                if (memory.completedObjectives.includes(index)) {
                    div.className += ' completed';
                    div.innerHTML = `✓ ${index + 1}. ${objective}`;
                } else if (index === memory.currentObjective) {
                    div.className += ' current';
                    div.innerHTML = `► ${index + 1}. ${objective}`;
                } else {
                    div.className += ' pending';
                    div.innerHTML = `${index + 1}. ${objective}`;
                }

                objectivesList.appendChild(div);
            });
        }

        async function sendMessage() {
            const message = document.getElementById('message').value.trim();
            if (!message) return;

            const chatDiv = document.getElementById('chat');
            chatDiv.innerHTML += `<p><b>You:</b> ${message}</p>`;
            document.getElementById('message').value = '';

            addDebug('Sending message: ' + message);

            try {
                const requestData = {
                    message: message,
                    memory: memory
                };

                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify(requestData)
                });

                if (!response.ok) {
                    throw new Error(`HTTP ${response.status}: ${response.statusText}`);
                }

                const data = await response.json();

                if (data.response) {
                    // Trigger talking animation
                    if (window.setAvatarTalking) {
                        window.setAvatarTalking(true);
                        setTimeout(() => window.setAvatarTalking(false), 2000);
                    }

                    chatDiv.innerHTML += `<p><b>Lieutenant:</b> ${data.response}</p>`;

                    if (data.memory) {
                        memory = data.memory;
                        updateMissionStatus();
                        addDebug('Mission status updated. Current objective: ' + memory.currentObjective);
                    }

                    if (data.stats) {
                        userStats = data.stats;
                        updateUserStats();
                    }
                } else if (data.error) {
                    chatDiv.innerHTML += `<p><b>System:</b> Error: ${data.error}</p>`;
                } else {
                    chatDiv.innerHTML += `<p><b>System:</b> No response received</p>`;
                }

            } catch (error) {
                addDebug('Error: ' + error.message);
                chatDiv.innerHTML += `<p><b>System:</b> Connection error: ${error.message}</p>`;
            }

            chatDiv.scrollTop = chatDiv.scrollHeight;
        }

        async function initApp() {
            await loadUserData();
            document.getElementById('chat').innerHTML += `<p><b>Lieutenant:</b> Oi, ${memory.name}, chummer! We got a job—grab Dr. Eva from the Towers. You in or you drek?</p>`;
            addDebug('Initialized with name: ' + memory.name + ', Current objective: ' + memory.currentObjective);
        }

        initApp();
    </script>
</body>
</html>
'''

# Test API before starting.
print("Testing API connection...")
api_working = test_api()

if not api_working:
    print("⚠️  WARNING: API test failed. Check your API key and model name.")

# Run Flask app.
def run_app():
    print("Starting Flask app...")
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

def main():
    flask_thread = threading.Thread(target=run_app)
    flask_thread.daemon = True
    flask_thread.start()

    # Wait for Flask to start.
    print("Waiting for Flask to start...")
    flask_ready = False
    for i in range(60):
        try:
            response = requests.get('http://localhost:5000/', timeout=1)
            if response.status_code == 200:
                print(f"✓ Flask is ready after {i * 0.5} seconds")
                flask_ready = True
                break
        except:
            pass
        time.sleep(0.5)
        if i % 10 == 0:
            print(f"  Still waiting for Flask... ({i * 0.5}s elapsed)")

    if not flask_ready:
        print("❌ Flask failed to start within 30 seconds")
        return

    # Set up ngrok.
    print("Setting up ngrok...")
    ngrok.set_auth_token("YOUR_KEY_HERE")  # Replace with your token.
    ngrok.kill()

    try:
        public_url = ngrok.connect(5000)
        print(f"🚀 Public URL: {public_url}")
        print("Click the link above to access your chatbot!")

        try:
            test_response = requests.get(f"{public_url}/", timeout=5)
            if test_response.status_code == 200:
                print("✓ Ngrok tunnel is working correctly!")
            else:
                print(f"⚠️  Ngrok tunnel connected but returned status {test_response.status_code}")
        except Exception as test_error:
            print(f"⚠️  Could not test ngrok tunnel: {test_error}")

    except Exception as ngrok_error:
        print(f"❌ Error setting up ngrok: {ngrok_error}")

    # Keep running.
    try:
        print("Server is running. Press Ctrl+C to stop.")
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Shutting down...")
        ngrok.kill()

if __name__ == "__main__":
    main()